# 推理配置自动搜索

千帆平台上的模型提供了大量参数可供用户调整，而这些参数设置会直接影响到模型表现，针对这个问题，SDK 提供了推理配置自动搜索功能，可以根据目标场景自动搜索出最佳的超参数配置。

首先我们需要准备一个 Evaluator，用于评估搜索的结果的好坏。

In [3]:
from qianfan.evaluation.evaluator import LocalEvaluator
from qianfan import ChatCompletion
from qianfan.common.prompt.prompt import Prompt
from qianfan.utils.pydantic import Field

from typing import Optional, Union, Any, Dict, List
import re
import json

class LocalJudgeEvaluator(LocalEvaluator):
    model: Optional[ChatCompletion] = Field(default=None, description="model object")
    metric_name: str = Field(default="", description="metric name for evaluation")
    eval_prompt: Prompt = Field(
        default=Prompt(
            template="""你需要扮演一个裁判的角色，对一段角色扮演的对话内容进行打分，你需要考虑这段文本中的角色沉浸度和对话文本的通畅程度。你可以根据以下规则来进行打分，你可以阐述你对打分标准的理解后再给出分数：
"4":完全可以扮演提问中的角色进行对话，回答完全符合角色口吻和身份，文本流畅语句通顺
"3":扮演了提问中正确的角色，回答完全符合角色口吻和身份，但文本不流畅或字数不满足要求
"2":扮演了提问中正确的角色，但是部分语句不符合角色口吻和身份，文本流畅语句通顺
"1":能够以角色的口吻和身份进行一部分对话，和角色设定有一定偏差，回答内容不流畅，或不满足文本字数要求
"0":扮演了错误的角色，没有扮演正确的角色，角色设定和提问设定差异极大，完全不满意
你的回答需要以json代码格式输出：
```json
{"modelA": {"justification": "此处阐述对打分标准的理解", "score": "此处填写打分结果"}}
```

现在你可以开始回答了：
问题：{{input}}
---
modelA回答：{{output}}
---""",
            identifier="{{}}",
        ),
        description="evaluation prompt",
    )

    class Config:
        arbitrary_types_allowed = True

    def evaluate(
        self, input: Union[str, List[Dict[str, Any]]], reference: str, output: str
    ) -> Dict[str, Any]:
        score = 0
        try:
            p, _ = self.eval_prompt.render(
                **{
                    "input": "\n".join([i["content"] for i in input[1:]]),
                    "output": output,
                    "expect": reference,
                }
            )
            r = self.model.do(messages=[{"role": "user", "content": p}])
            content = r["result"]
            regex = re.compile("\`\`\`json(.*)\`\`\`", re.MULTILINE | re.DOTALL)

            u = regex.findall(content)

            if len(u) == 0:
                score = 0
            else:
                score = float(json.loads(u[0])["modelA"]["score"])
        except Exception as e:
            score = 0
        return {self.metric_name: score}

超参搜索主要分为两部分：

- `Suggestor`：超参搜索算法，负责从搜索空间中选取一组配置
- `Runner`：运行器，负责根据 `Suggestor` 提供的配置运行模型并给出结果

SDK 已经内置了 `Suggestor` 和 `Runner`，用户只需要提供搜索空间和待评估的数据集和评估器即可。

In [ ]:
from qianfan.autotuner.launcher import Launcher
from qianfan.autotuner.suggestor import RandomSuggestor
from qianfan.autotuner.runner import QianfanRunner
from qianfan.autotuner.space import Uniform, Categorical
from qianfan.dataset import Dataset

context = await Launcher().run(
    suggestor=RandomSuggestor(
        search_space = {
            "temperature": Uniform(0.01, 0.99),  # 设定temperature的范围
            "model": Categorical(["ERNIE-Speed"]),  # 设定model的取值范围
        },
        cost_budget=0.001,  # 设定整个流程的预算
    ),
    runner=QianfanRunner(
        dataset=Dataset.load(
            data_file="./example.jsonl",
            organize_data_as_group=False,
            input_columns=["prompt"],
            reference_column="response",
        ),
        evaluator=LocalJudgeEvaluator(
            model=ChatCompletion(model="ERNIE-Bot-4"), metric_name="accuracy"
        ),
    ),
)


返回的结果是一个 `Context` 对象，其中包含了整个搜索过程的所有上下文信息，例如可以通过如下方式获得搜索的最佳参数

In [5]:
context.best

{'temperature': 0.49294604892967614, 'model': 'ERNIE-Speed'}

也可以获取某一轮某一组配置的评估结果等信息

In [8]:
context.history[0][0].metrics

{'accuracy': 3.1,
 'avg_prompt_tokens': 648.3,
 'avg_completion_tokens': 201.3,
 'avg_total_tokens': 849.6,
 'avg_req_latency': 4.48412966793403,
 'avg_tokens_per_second': 189.46820518493962,
 'avg_cost': 0.0042036,
 'total_cost': 0.042036000000000004}